In [ ]:
#default_exp PracticalDeepLearning.Lesson1.CourseWork

In [ ]:
#hide
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# Lesson 1
> Image Classification using Fastaiv2

Notes following the first lesson of the fastai 2019 Deep Learning for Practitioners Course. Code follows the lesson using fastaiv2, fastcore and is documented using the nbdev library.